MLlib is a Spark subproject providing machine learning algorithm. 

In [1]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
 df = sqlContext.read.format('com.databricks.spark.csv').option("header", 'true').load("s3://analytic-partners-sparktest/test.csv")

In [4]:
df.count()

10000000

In [5]:
from pyspark.sql.functions import col  # for indicating a column using a string in the line below
df = df.select([col(c).cast("double").alias(c) for c in df.columns])
df.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- X1: double (nullable = true)
 |-- X2: double (nullable = true)
 |-- X3: double (nullable = true)
 |-- X4: double (nullable = true)
 |-- X5: double (nullable = true)
 |-- X6: double (nullable = true)
 |-- X7: double (nullable = true)
 |-- X8: double (nullable = true)
 |-- X9: double (nullable = true)
 |-- X10: double (nullable = true)
 |-- Y: double (nullable = true)



In [6]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer
featuresCols = df.columns
featuresCols.remove('Y')
# This concatenates all feature columns into a single feature vector in a new column "rawFeatures".
vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="features")


In [37]:
train=vectorAssembler.transform(df)

In [38]:
train

DataFrame[_c0: double, X1: double, X2: double, X3: double, X4: double, X5: double, X6: double, X7: double, X8: double, X9: double, X10: double, Y: double, features: vector]

In [ ]:
def parse_points(df):
    label = float(df[0][0])
    features = list(map(float,df[0][1:]))
    return LabeledPoint(label,features)


train1=df.rdd.map(lambda x: parse_points(x))

In [49]:
import time
start_time = time.time()
model = LinearRegressionWithSGD.train(train1, iterations=100, step=0.000001,convergenceTol=0.0001,intercept=True)
print("--- %s seconds ---" % (time.time() - start_time))

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 25.0 failed 4 times, most recent failure: Lost task 0.3 in stage 25.0 (TID 138, ip-172-31-41-253.ec2.internal): org.apache.spark.SparkException: 
Error from python worker:
  /usr/bin/python: No module named pyspark
PYTHONPATH was:
  /mnt1/yarn/usercache/hadoop/filecache/19/__spark_libs__613031940707385496.zip/spark-core_2.11-2.0.2.jar
java.io.EOFException
	at java.io.DataInputStream.readInt(DataInputStream.java:392)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:166)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:89)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:65)
	at org.apache.spark.SparkEnv.createPythonWorke

In [42]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(maxIter=10, regParam=0, elasticNetParam=0,labelCol="Y")

In [45]:
import time
start_time = time.time()
lrModel = lr.fit(train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 42.4667692184 seconds ---


In [46]:
lrModel.coefficients

[5.46883446606e-08,1.00887883136,1.97718134351,2.99673297562,3.99057874219,4.98239140199,6.00236176012,7.04116530944,7.92453145472,8.94738609245,10.0383600564]

In [47]:
trainingSummary = lrModel.summary

In [48]:
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 999.980337
r2: 0.036959
